In [ ]:
# Adapted from example notebooks provided at: https://alleninstitute.github.io/abc_atlas_access/intro.html

import pandas as pd
import numpy as np
import anndata
from pathlib import Path
import matplotlib.pyplot as plt
import time

from abc_atlas_access.abc_atlas_cache.abc_project_cache import AbcProjectCache

In [ ]:
basePath = 'Z:\\Common\\Transcriptomics\\ABC_Atlas' # Change to relevant folder path
download_base = Path(basePath)
abc_cache = AbcProjectCache.from_cache_dir(download_base)

abc_cache.current_manifest

In [ ]:
cell = abc_cache.get_metadata_dataframe(directory='WMB-10X', file_name='cell_metadata_with_cluster_annotation')
cell.set_index('cell_label',inplace=True)

In [ ]:
matrices = cell.groupby(['dataset_label', 'feature_matrix_label'])[['library_label']].count()
matrices.columns  = ['cell_count']
matrices

In [ ]:
abc_cache.list_data_files('WMB-10XMulti')

In [ ]:
file = abc_cache.get_data_path(directory='WMB-10XMulti', file_name='WMB-10XMulti/log2')
print(file)

In [ ]:
ad = anndata.read_h5ad(file,backed='r')
gene = ad.var

In [ ]:
df = pd.read_csv(basePath + '\\input\\WMB_genes_Hb_new.csv')
gn = df['gene_symbol'].tolist()
pred = [x in gn for x in gene.gene_symbol]
gene_filtered = gene[pred]
print("Number of selected genes = ", len(gn))
gene_filtered

In [ ]:
# create empty gene expression dataframe
gdata = pd.DataFrame(index=cell.index, columns=gene_filtered.index)
count = 0
total_start = time.process_time()

for matindex in matrices.index:
    
    ds = matindex[0]
    mp = matindex[1]
    
    print(mp)
    
    file = abc_cache.get_data_path(directory=ds, file_name=mp + '/log2')
    
    pred = (cell['feature_matrix_label'] == mp)
    cell_filtered = cell[pred]
    
    start = time.process_time()
    ad = anndata.read_h5ad(file, backed='r')
    exp = ad[cell_filtered.index, gene_filtered.index].to_df()
    gdata.loc[ exp.index, gene_filtered.index ] = exp
    print(" - time taken: ", time.process_time() - start)
    
    ad.file.close()
    del ad
    
    count += 1
    
#    if count > 2 :
#       break
        
print("total time taken: ", time.process_time() - total_start)
    

In [ ]:
# change columns from index to gene symbol
gdata.columns = gene_filtered.gene_symbol
pred = pd.notna(gdata[gdata.columns[0]])
gdata = gdata[pred].copy(deep=True)
gdata = gdata[gn]
print(len(gdata))
gdata

In [ ]:
def aggregate_by_metadata(df, gnames, value, sort = False):
    grouped = df.groupby(value)[gn].mean()
    if sort:
        grouped = grouped.sort_values(by=gn[0], ascending=False)
    return grouped

In [ ]:
def plot_umap(xx, yy, cc=None, val=None, fig_width=8, fig_height=8, cmap=None):
    
    fig, ax = plt.subplots()
    fig.set_size_inches(fig_width, fig_height)
    
    if cmap is not None :
        plt.scatter(xx, yy, s=0.5, c=val, marker='.', cmap=cmap)
    elif cc is not None :
        plt.scatter(xx, yy, s=0.5, color=cc, marker='.')
        
    ax.axis('equal')
    ax.set_xlim(-18, 27)
    ax.set_ylim(-18, 27)
    ax.set_xticks([])
    ax.set_yticks([])
    
    return fig, ax

In [ ]:
def plot_heatmap(df, fig_width=8, fig_height=4, cmap=plt.cm.magma_r):

    arr = df.to_numpy(dtype='float')

    fig, ax = plt.subplots()
    fig.set_size_inches(fig_width, fig_height)

    im = ax.imshow(arr, cmap=cmap, aspect='auto', vmin=0, vmax=6)
    xlabs = df.columns.values
    ylabs = df.index.values

    ax.set_xticks(range(len(xlabs)))
    ax.set_xticklabels(xlabs)

    ax.set_yticks(range(len(ylabs)))
    res = ax.set_yticklabels(ylabs)
    
    return im

In [ ]:
# Filter cell dataframe to cells of interest
cs = ["17 MH-LH Glut"] # class
CL = cell[cell['class'].isin(cs)]
print("Number of cells = ", len(CL))
CL.head(5)

In [ ]:
# Filter gdata dataframe to cells of interest
GD = gdata[cell['class'].isin(cs)]
print("Number of cells = ", len(GD))
GD.head(5)

In [ ]:
# Export metadata and gene expression:
CL.to_csv(basePath + '\\output\\Hb-new_CL_metadata.csv')
GD.to_csv(basePath + '\\output\\Hb-new_GD_expression.csv') 